#BERT Classifier

In [1]:
import torch, os

In [2]:
!pip install transformers dataset

In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [5]:
from google.colab import files
movies = files.upload()

Saving movies.csv to movies.csv


In [16]:
import pandas as pd
col_names = ["genre", "split"]
data = pd.read_csv("movies.csv", sep=";", names=col_names)
data = data.sample(frac=1, random_state=42)
data.head()

,genre,split
8826,superhero,"ruling, brother. A throne would suit you ill. ..."
3172,comedy,the hotel. You'd better go straight to the bas...
9221,western,"shooting a man, son. No, it isn't. Not in my o..."
718,action,that. We've only got seven minutes. Passengers...
3133,comedy,The Thames Men later became Spinal Tap and had...


In [17]:
labels = ["action", "adventure", "comedy", "drama", "fantasy", "history", "scifi", "sport", "superhero", "western"]
NUM_LABELS = len(labels)
id2label = {i:l for i,l in enumerate(labels)}
label2id = {l:i for i,l in enumerate(labels)}

In [18]:
label2id

{'action': 0,
 'adventure': 1,
 'comedy': 2,
 'drama': 3,
 'fantasy': 4,
 'history': 5,
 'scifi': 6,
 'sport': 7,
 'superhero': 8,
 'western': 9}

In [19]:
data["labels"] = data.genre.map(lambda x: label2id[x.strip()])

In [20]:
data.head()

,genre,split,labels
8826,superhero,"ruling, brother. A throne would suit you ill. ...",8
3172,comedy,the hotel. You'd better go straight to the bas...,2
9221,western,"shooting a man, son. No, it isn't. Not in my o...",9
718,action,that. We've only got seven minutes. Passengers...,0
3133,comedy,The Thames Men later became Spinal Tap and had...,2
